In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import requests
import numpy as np
import math
import random as ran
import torch
import torch.nn as nn 
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F
import matplotlib.pyplot as plt 
import sklearn.metrics
import random

In [ ]:
train_dataset = torchvision.datasets.MNIST(root='./data', train=True, transform=transforms.ToTensor(),   download=True)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=1,  shuffle=True)
examples = iter(train_loader)
data,targets = examples.next()

In [ ]:
test_dataset = torchvision.datasets.MNIST(root='./data', train=False, transform=transforms.ToTensor(),download=True)

In [ ]:
def load(train_d):
  data=[]
  for i in range(10):
    count=0
    ind=0
    img_lst=[]
    while(True):

      if train_d[ind][1]==i:
        img_lst.append(train_d[ind][0].reshape(1,1,28,28))
        count+=1
      if count==100:
        data.append(img_lst)
        break
      ind+=1
  return data

In [ ]:
def load_test(test_d):
  data=[]
  for i in range(10):
    count=0
    ind=0
    img_lst=[]
    while(True):
      if test_d[ind][1]==i:
        img_lst.append(test_d[ind][0].reshape(1,1,28,28))
        count+=1
      if count==10:
        data.append(img_lst)
        break
      ind+=1
  return data

In [ ]:
data=load(train_dataset)
test=load_test(test_dataset)

In [ ]:
img=data[0][0]
t=img.shape[0]
img=img.reshape((1,1,img.shape[0],img.shape[0]))
print(img.shape)

torch.Size([1, 1, 28, 28])


In [ ]:
def one_hot_encode(obj):
  num=obj
  arr_z=np.zeros(10)
  arr_z[num]=1
  arr_z=torch.tensor(arr_z)
  return arr_z

# Q 1

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1,4,5, padding=2, stride=1)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(4,4,5, padding=2, stride=1)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(196, 49)
        self.fc2 = nn.Linear(49,10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool1(x)
        x = F.relu(self.conv2(x))
        x = self.pool2(x)
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        return x

In [ ]:
net = Net()
out=net(img)
enc_target=one_hot_encode(example_targets[0]).float().reshape(1,10)
enc_target.shape

torch.Size([1, 10])

In [ ]:
print("Softmax output is ", out)

Softmax output is  tensor([[0.0000, 0.0285, 0.0000, 0.1427, 0.0172, 0.0000, 0.0766, 0.0356, 0.0000,
         0.0000]], grad_fn=<ReluBackward0>)


In [ ]:
cross_entropy_loss = nn.CrossEntropyLoss()
output = cross_entropy_loss(out, enc_target)
output.backward()
print("Loss ",output.item())

Loss  2.333660125732422


# Q 2

## 2a) Vanilla SGD

In [ ]:
epochs=15
cross_entropy_loss = nn.CrossEntropyLoss()
net = Net()
sgd_optimizer = optim.SGD(net.parameters(), lr=0.001)
loss_lst_plot=[]
for _ in range(epochs):
  loss_lst=[]

  for shuffle_ind in range(10):
    random.shuffle(data[shuffle_ind])
  for ind1 in range(10):
    sgd_optimizer.zero_grad()
    for ind2 in range (100):
      img=data[ind1][ind2].reshape(1,1,28,28)
      out=net(img)
      enc_target=one_hot_encode(ind1).float().reshape(1,10)    
      cross_loss = cross_entropy_loss(out, enc_target)
      
      cross_loss.backward()
      
      loss_lst.append(cross_loss.item())
    sgd_optimizer.step()
  loss=sum(loss_lst)/len(loss_lst)
  loss_lst_plot.append(loss)
  print(loss)


2.3058513717651365
2.3026286091804504
2.302596381187439
2.302589612007141
2.3025857136249543
2.302583212375641
2.3025811629295347
2.3025789728164674
2.302577027320862
2.3025764274597167
2.3025744841098787
2.3025702283382414
2.3025676963329316
2.3025699365139007
2.302567396879196


In [ ]:
plt.plot(loss_lst_plot)
plt.title("SGD")
plt.show()


## 2b) Momentum

In [ ]:
epochs=15
cross_entropy_loss = nn.CrossEntropyLoss()
net = Net()
mom_optimizer = optim.SGD(net.parameters(), lr=0.001,momentum=0.9)
loss_lst_plot_mom=[]
for _ in range(epochs):
  loss_lst=[]

  for shuffle_ind in range(10):
    random.shuffle(data[shuffle_ind])
  for ind1 in range(10):
    mom_optimizer.zero_grad()
    for ind2 in range (100):
      img=data[ind1][ind2].reshape(1,1,28,28)
      out=net(img)
      enc_target=one_hot_encode(ind1).float().reshape(1,10)    
      cross_loss = cross_entropy_loss(out, enc_target)
      
      cross_loss.backward()
      
      loss_lst.append(cross_loss.item())
    mom_optimizer.step()
  loss=sum(loss_lst)/len(loss_lst)
  loss_lst_plot_mom.append(loss)
  print(loss)

In [ ]:
plt.plot(loss_lst_plot_mom)
plt.title("Momentum")
plt.show()

## 2c) RMSprop

In [ ]:
epochs=15
cross_entropy_loss = nn.CrossEntropyLoss()
net = Net()
rms_optimizer = optim.RMSprop(net.parameters(), lr=0.001,alpha=0.9)
loss_lst_plot_rms=[]
for _ in range(epochs):
  loss_lst=[]
  for shuffle_ind in range(10):
    random.shuffle(data[shuffle_ind])
  for ind1 in range(10):
   
    rms_optimizer.zero_grad()
    for ind2 in range (100):
      img=data[ind1][ind2].reshape(1,1,28,28)
      out=net(img)
      enc_target=one_hot_encode(ind1).float().reshape(1,10)    
      cross_loss = cross_entropy_loss(out, enc_target)
      
      cross_loss.backward()
      
      loss_lst.append(cross_loss.item())
    rms_optimizer.step()
  loss=sum(loss_lst)/len(loss_lst)
  loss_lst_plot_rms.append(loss)
  print(loss)

In [ ]:
plt.plot(loss_lst_plot_rms)
plt.title("RMSProp")
plt.show()

In [ ]:
class Net_1(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1,4,5, padding=2, stride=1)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(4,4,5, padding=2, stride=1)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(196, 49)
        self.fc2 = nn.Linear(49,10)

    def forward(self, x):
        x_act1 = F.relu(self.conv1(x))
        x = self.pool1(x_act1)
        x_act2 = F.relu(self.conv2(x))
        x = self.pool2(x_act2)
        x_flat = torch.flatten(x, 1)
        x = F.relu(self.fc1(x_flat))
        x = F.relu(self.fc2(x))
        return x,x_act1,x_act2,x_flat

# Q 3

## 3a) Train and Test error

In [ ]:
epochs=100
ee=1
cross_entropy_loss = nn.CrossEntropyLoss()
net = Net_1()
rms_optimizer = optim.RMSprop(net.parameters(), lr=0.001,alpha=0.9)
loss_lst_plot_rms=[]
test_loss_plot=[]
train_error=[]
test_error=[]
act_1=[]
act_2=[]
flat=[]
for _ in range(epochs):
  loss_lst=[]
  test_loss_lst=[]

  for ind1 in range(10):
    net.zero_grad()
    y_pred=[]
    
    img=torch.cat(data[ind1],dim=0)
    out,t1,t2,t3=net(img)
    act_1.append(t1)
    act_2.append(t2)
    flat.append(t3)
    enc_target=one_hot_encode(ind1).float().reshape(1,10).tolist()
    enc_arr=[ind1]*100    
    cross_loss = cross_entropy_loss(out, torch.tensor(enc_arr))
    cross_loss.backward()
    loss_lst.append(cross_loss.item())
    rms_optimizer.step()
    for i in range(100):
      if out[i].argmax()== ind1:
        y_pred.append(True)
      else:
        y_pred.append(False)
  y_true=[True]*100
  loss=sum(loss_lst)/len(loss_lst)
  loss_lst_plot_rms.append(loss)
  acc_lst=[]
  acc_lst_train=[]
  
  print("Training loss",loss)

  acc_lst_train.append(sklearn.metrics.accuracy_score(y_pred,y_true))
  acc_train=sum(acc_lst_train)/len(acc_lst_train)

  for ind1 in range(10):

    img=torch.cat(test[ind1],dim=0)
    out,_,_,_=net(img)
    enc_target=one_hot_encode(ind1).float().reshape(1,10)
    enc_arr=[ind1]*10    
    cross_loss = cross_entropy_loss(out,torch.tensor( enc_arr))
    y_pred=[]
    test_loss_lst.append(cross_loss.item())
    for i in range(10):
      if out[i].argmax()== ind1:
        y_pred.append(True)
      else:
        y_pred.append(False)
    y_true=[True]*10

    acc_lst.append(sklearn.metrics.accuracy_score(y_pred,y_true))
  loss=sum(test_loss_lst)/len(test_loss_lst)
  acc=sum(acc_lst)/len(acc_lst)
  test_loss_plot.append(loss)
  train_error.append(1-acc_train)
  test_error.append(1-acc)
  print("Epoch",ee)
  ee+=1
  print("Test loss",loss)
  print("Training Accuracy is",acc_train)
  print("Testing Accuracy is ",acc)
  print("_________________________")

In [ ]:
plt.plot(loss_lst_plot_rms,label='train_loss')
plt.plot(test_loss_plot,label='test_loss')
plt.title("RMSProp")
plt.legend()
plt.show()

In [ ]:
plt.plot(train_error,label="train")
plt.plot(test_error,label="test")
plt.title("Test_error vs Train_error")
plt.legend()
plt.show()

## 3b) Visualizing activation maps

In [ ]:
plt.figure(figsize=(20,10))
for i in range(15):
  ind=random.randint(0,14)
  plt.subplot(1,15,i+1)
  plt.imshow(act_1[ind][1][1].detach().numpy())
plt.show()

## 3c) Final Accuracy

In [ ]:
print("The final accuracy of the model is ",acc*100)

# (4) TSNE plots

In [ ]:
len(flat)

In [ ]:
from sklearn.manifold import TSNE
X=TSNE(n_components=2,perplexity=20,init='random').fit_transform(flat[0].detach().numpy())


In [ ]:
for i in range(len(X)):
  plt.scatter(X[i][0],X[i][1])
plt.show()

TSNE plot for first epoch


In [ ]:
X=TSNE(n_components=2,perplexity=20,init='random').fit_transform(flat[999].detach().numpy())
for i in range(len(X)):
  plt.scatter(X[i][0],X[i][1])
plt.show()

TSNE plot for last epoch


# (6) Dropout

In [ ]:
class Net_2(nn.Module):
    def __init__(self,drop):
        super().__init__()
        self.conv1 = nn.Conv2d(1,4,5, padding=2, stride=1)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(4,4,5, padding=2, stride=1)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(196, 49)
        self.fc2 = nn.Linear(49,10)
        self.dropout = nn.Dropout(drop)

    def forward(self, x):
        x_act1 = F.relu(self.conv1(x))
        x = self.pool1(x_act1)
        x_act2 = F.relu(self.conv2(x))
        x = self.pool2(x_act2)
        x_flat = torch.flatten(x, 1)
        x = F.relu(self.fc1(x_flat))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
      
        return x,x_act1,x_act2,x_flat

In [ ]:
epochs=45
cross_entropy_loss = nn.CrossEntropyLoss()
drop=[0.2,0.5,0.8]
for d in drop:
  print("for Dropout = ",d)
  net = Net_2(d)
  rms_optimizer = optim.RMSprop(net.parameters(), lr=0.001,alpha=0.9)
  loss_lst_plot_rms=[]
  test_loss_plot=[]
  train_error=[]
  test_error=[]
  act_1=[]
  act_2=[]
  flat=[]
  for _ in range(epochs):
    loss_lst=[]
    test_loss_lst=[]
    for ind1 in range(10):
      net.zero_grad()
      y_pred=[]
      
      img=torch.cat(data[ind1],dim=0)
      out,t1,t2,t3=net(img)
      act_1.append(t1)
      act_2.append(t2)
      flat.append(t3)
      enc_arr=[ind1]*100    
      cross_loss = cross_entropy_loss(out, torch.tensor(enc_arr))
      cross_loss.backward()
      loss_lst.append(cross_loss.item())
      rms_optimizer.step()
      for i in range(100):
        if out[i].argmax()== ind1:
          y_pred.append(True)
        else:
          y_pred.append(False)
    y_true=[True]*100
    loss=sum(loss_lst)/len(loss_lst)
    loss_lst_plot_rms.append(loss)
    acc_lst=[]
    acc_lst_train=[]

    acc_lst_train.append(sklearn.metrics.accuracy_score(y_pred,y_true))
    acc_train=sum(acc_lst_train)/len(acc_lst_train)

    for ind1 in range(10):

      img=torch.cat(test[ind1],dim=0)
      out,_,_,_=net(img)
      enc_arr=[ind1]*10    
      cross_loss = cross_entropy_loss(out,torch.tensor( enc_arr))
      y_pred=[]
      test_loss_lst.append(cross_loss.item())
      for i in range(10):
        if out[i].argmax()== ind1:
          y_pred.append(True)
        else:
          y_pred.append(False)
      y_true=[True]*10

      acc_lst.append(sklearn.metrics.accuracy_score(y_pred,y_true))
    loss=sum(test_loss_lst)/len(test_loss_lst)
    acc=sum(acc_lst)/len(acc_lst)
    test_loss_plot.append(loss)
    train_error.append(1-acc_train)
    test_error.append(1-acc)
  print("Training Accuracy is",acc_train)
  print("Testing Accuracy is ",acc)
  print("___________________________")

# (7)Batch Norm

In [ ]:
class Net_3(nn.Module):
    def __init__(self,drop):
        super().__init__()
        self.conv1 = nn.Conv2d(1,4,5, padding=2, stride=1)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(4,4,5, padding=2, stride=1)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(196, 49)
        self.fc2 = nn.Linear(49,10)
        self.dropout = nn.Dropout(drop)
        self.batch = nn.BatchNorm1d(49)

    def forward(self, x):
        x_act1 = F.relu(self.conv1(x))
        x = self.pool1(x_act1)
        x_act2 = F.relu(self.conv2(x))
        x = self.pool2(x_act2)
        x_flat = torch.flatten(x, 1)
        x = F.relu(self.fc1(x_flat))
        x = self.dropout(x)
        x = self.batch()
        x = F.relu(self.fc2(x))
        
        return x,x_act1,x_act2,x_flat

In [ ]:
epochs=100
cross_entropy_loss = nn.CrossEntropyLoss()
drop=[0.2,0.5,0.8]
print("Batch Normalization and dropout")
for d in drop:
  print("for Dropout ",d)
  net = Net_2(d)
  rms_optimizer = optim.RMSprop(net.parameters(), lr=0.001,alpha=0.9)
  loss_lst_plot_rms=[]
  test_loss_plot=[]
  train_error=[]
  test_error=[]
  act_1=[]
  act_2=[]
  flat=[]
  for _ in range(epochs):
    loss_lst=[]
    test_loss_lst=[]
    for ind1 in range(10):
      net.zero_grad()
      y_pred=[]
      
      img=torch.cat(data[ind1],dim=0)
      out,t1,t2,t3=net(img)
      act_1.append(t1)
      act_2.append(t2)
      flat.append(t3)
      enc_arr=[ind1]*100    
      cross_loss = cross_entropy_loss(out, torch.tensor(enc_arr))
      cross_loss.backward()
      loss_lst.append(cross_loss.item())
      rms_optimizer.step()
      for i in range(100):
        if out[i].argmax()== ind1:
          y_pred.append(True)
        else:
          y_pred.append(False)
    y_true=[True]*100
    loss=sum(loss_lst)/len(loss_lst)
    loss_lst_plot_rms.append(loss)
    acc_lst=[]
    acc_lst_train=[]
    acc_lst_train.append(sklearn.metrics.accuracy_score(y_pred,y_true))
    acc_train=sum(acc_lst_train)/len(acc_lst_train)
    
    for ind1 in range(10):

      img=torch.cat(test[ind1],dim=0)
      out,_,_,_=net(img)
      enc_arr=[ind1]*10    
      cross_loss = cross_entropy_loss(out,torch.tensor( enc_arr))
      y_pred=[]
      test_loss_lst.append(cross_loss.item())
      for i in range(10):
        if out[i].argmax()== ind1:
          y_pred.append(True)
        else:
          y_pred.append(False)
      y_true=[True]*10

      acc_lst.append(sklearn.metrics.accuracy_score(y_pred,y_true))
    loss=sum(test_loss_lst)/len(test_loss_lst)
    acc=sum(acc_lst)/len(acc_lst)
    test_loss_plot.append(loss)
    train_error.append(1-acc_train)
    test_error.append(1-acc)

  print("Training Accuracy is",acc_train)
  print("Testing Accuracy is ",acc)